In [24]:
import lkml as looker
from pprint import pprint
from google.oauth2 import service_account
import pandas_gbq
from contextlib import redirect_stdout
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import pandas as pd
import pandas
import os
import json
import sys
import yaml

import warehouse_target
import config

import git

pd.options.mode.chained_assignment = None

credentials = config.service_account

warehouse_name = config.warehouse_name
lookml_project = config.project_name

sql = warehouse_target.warehouse_schema
explore_sql = warehouse_target.explore_sql

if warehouse_name == 'big_query':

    # Run a Standard SQL query with the project set explicitly
    project_id = lookml_project
    df = pandas.read_gbq(sql, dialect='standard', project_id=lookml_project, credentials=credentials)

    df['description'] = df['description'].fillna('not available')

    df1 = df[['table_name','column_name','data_type','description']]

    df1['data_type'] = df1['data_type'].str.replace('TIMESTAMP','timestamp')
    df1['data_type'] = df1['data_type'].str.replace('DATE','date')
    df1['data_type'] = df1['data_type'].str.replace('INT64','number')
    df1['data_type'] = df1['data_type'].str.replace('FLOAT64','number')
    df1['data_type'] = df1['data_type'].str.replace('NUMERIC','number')
    df1['data_type'] = df1['data_type'].str.replace('STRING','string')
    df1['data_type'] = df1['data_type'].str.replace('BOOL','yesno')

    df2 = df1

    explore_df = pandas.read_gbq(explore_sql, dialect='standard', project_id=lookml_project, credentials=credentials)
  
    explore_df_2 = explore_df[['parent_table_name','pk_table_name', 'pk_column_name','fk_table_name','fk_column_name','looker_relationship']]


df4 = {n: grp.loc[n].to_dict('index')
    
for n, grp in explore_df_2.set_index(['parent_table_name','pk_table_name', 'pk_column_name','fk_table_name','fk_column_name','looker_relationship']).groupby(level='parent_table_name')}

d2 = df4

##


explore_relationship_df = explore_df[['looker_relationship']]

relationship_dict = explore_relationship_df.to_dict('list')



				

In [25]:
d2

{'sales_applications': {('core_users',
   'user_pk',
   'core_user_mapping',
   'user_fk',
   'many_to_one'): {},
  ('core_institutions',
   'institution_pk',
   'product_progression_plans',
   'institution_fk',
   'many_to_one'): {},
  ('core_institutions',
   'institution_pk',
   'core_programs',
   'institution_fk',
   'many_to_one'): {},
  ('core_institutions',
   'institution_pk',
   'product_study_plans',
   'institution_fk',
   'many_to_one'): {},
  ('core_countries',
   'country_pk',
   'core_country_mapping',
   'country_fk',
   'many_to_one'): {},
  ('core_countries',
   'country_pk',
   'core_institutions',
   'country_fk',
   'many_to_one'): {},
  ('core_countries',
   'country_pk',
   'sales_student',
   'country_fk',
   'many_to_one'): {},
  ('core_countries',
   'country_pk',
   'core_agents',
   'country_fk',
   'many_to_one'): {},
  ('core_countries',
   'country_pk',
   'core_sponsors',
   'country_fk',
   'many_to_one'): {},
  ('core_agents',
   'agent_pk',
   'sales

In [29]:
def get_all_values(nested_dictionary):

    
    for key,value in nested_dictionary.items():

        explore = {

            "explore": key,

            "view_name": key
                
        }
            
        
        yield(looker.dump(explore))

        for key1 in value.keys():     

            join = {

                "joins": [
                    {
                    "sql_on": key1[0]+"."+key1[1]+" = "+ key1[2]+"."+key1[3],
                    "relationship": key1[4],
                    "name": key1[0]
                    }

                ]
                }

            yield(looker.dump(join))


nested_dictionary = d2

In [30]:
for value in get_all_values(nested_dictionary):

    print(value)

explore: sales_applications
view_name: sales_applications
join: core_users {
  sql_on: core_users.user_pk = core_user_mapping.user_fk ;;
  relationship: many_to_one
}
join: core_institutions {
  sql_on: core_institutions.institution_pk = product_progression_plans.institution_fk ;;
  relationship: many_to_one
}
join: core_institutions {
  sql_on: core_institutions.institution_pk = core_programs.institution_fk ;;
  relationship: many_to_one
}
join: core_institutions {
  sql_on: core_institutions.institution_pk = product_study_plans.institution_fk ;;
  relationship: many_to_one
}
join: core_countries {
  sql_on: core_countries.country_pk = core_country_mapping.country_fk ;;
  relationship: many_to_one
}
join: core_countries {
  sql_on: core_countries.country_pk = core_institutions.country_fk ;;
  relationship: many_to_one
}
join: core_countries {
  sql_on: core_countries.country_pk = sales_student.country_fk ;;
  relationship: many_to_one
}
join: core_countries {
  sql_on: core_countries.

In [31]:
d2

{'sales_applications': {('core_users',
   'user_pk',
   'core_user_mapping',
   'user_fk',
   'many_to_one'): {},
  ('core_institutions',
   'institution_pk',
   'product_progression_plans',
   'institution_fk',
   'many_to_one'): {},
  ('core_institutions',
   'institution_pk',
   'core_programs',
   'institution_fk',
   'many_to_one'): {},
  ('core_institutions',
   'institution_pk',
   'product_study_plans',
   'institution_fk',
   'many_to_one'): {},
  ('core_countries',
   'country_pk',
   'core_country_mapping',
   'country_fk',
   'many_to_one'): {},
  ('core_countries',
   'country_pk',
   'core_institutions',
   'country_fk',
   'many_to_one'): {},
  ('core_countries',
   'country_pk',
   'sales_student',
   'country_fk',
   'many_to_one'): {},
  ('core_countries',
   'country_pk',
   'core_agents',
   'country_fk',
   'many_to_one'): {},
  ('core_countries',
   'country_pk',
   'core_sponsors',
   'country_fk',
   'many_to_one'): {},
  ('core_agents',
   'agent_pk',
   'sales

In [34]:
type(d2)

dict

In [36]:
import collections

In [38]:
dictA = {'Sun': 5, 'Mon': 3, 'Tue': 5, 'Wed': 3}

print("Given Dictionary :", dictA)

k_v_exchanged = {}

for key, value in dictA.items():
   if value not in k_v_exchanged:
      k_v_exchanged[value] = [key]
   else:
      k_v_exchanged[value].append(key)

# Result
print("New Dictionary:", k_v_exchanged)

Given Dictionary : {'Sun': 5, 'Mon': 3, 'Tue': 5, 'Wed': 3}
New Dictionary: {5: ['Sun', 'Tue'], 3: ['Mon', 'Wed']}


In [40]:
k_v_exchanged = {}


for key, value in d2.items():
   if value not in k_v_exchanged:
      k_v_exchanged[value] = [key]
   else:
      k_v_exchanged[value].append(key)


TypeError: unhashable type: 'dict'

In [85]:
    for key,value in nested_dictionary.items():

        explore = {

            "explore": key,

            "view_name": key
                
        }
            
        print(explore)

        for key1 in value.keys():  

           ## if len(key1[0]) != len(set(key1[0])):

            ##if key1[0] in key1[0]:
            
            join = { "table":key1[0]}
                
              ##  print(join)
                
            print(join)

{'explore': 'sales_applications', 'view_name': 'sales_applications'}
{'table': 'core_users'}
{'table': 'core_institutions'}
{'table': 'core_institutions'}
{'table': 'core_institutions'}
{'table': 'core_countries'}
{'table': 'core_countries'}
{'table': 'core_countries'}
{'table': 'core_countries'}
{'table': 'core_countries'}
{'table': 'core_agents'}
{'table': 'product_study_plans'}
{'table': 'core_institutions'}
{'table': 'sales_applications'}
{'table': 'sales_applications'}
{'table': 'sales_applications'}
{'table': 'sales_student'}
{'table': 'core_sponsors'}
{'table': 'product_study_plans'}


In [81]:
flipped = {}

for key, value in d2.items():
    if value not in flipped:
        flipped[value] = [key]
    else:
        flipped[value].append(key)

TypeError: unhashable type: 'dict'

In [82]:
def get_dups(a, values=None):
    if values is None: values = []
    if (a in values): return True
    values.append(a)
    if type(a) == dict:
        for i in a.values():
            if (get_dups(i, values=values)):
                return True
    return False

print(get_dups(d2))

False


In [83]:
dictionary = d2
rev_dictionary = {}

for key, value in dictionary.items():
    rev_dictionary.setdefault(value, set()).add(key)
    print(rev_dictionary)

for key,values in dictionary.items():
    if len(values) > 1:
        values = True
    else:
        values = False

TypeError: unhashable type: 'dict'